In [10]:
%%writefile app_soporte.py

import streamlit as st
import requests
import json
import os

# ========================
# Configuración
# ========================
API_URL = "http://localhost:5000/rag"  # Cambia si tu Flask está en otro host o puerto
HISTORIAL_FILE = "historial_chat.json"  # Archivo donde se guarda el historial

# ========================
# Cargar historial desde disco (si existe)
# ========================
if os.path.exists(HISTORIAL_FILE):
    with open(HISTORIAL_FILE, "r", encoding="utf-8") as f:
        st.session_state.chat_history = json.load(f)
else:
    st.session_state.chat_history = []

# ========================
# Función para enviar pregunta a Flask
# ========================
def send_question(question, chat_history):
    payload = {
        "question": question,
        "chat_history": "\n".join([m["text"] for m in chat_history if m["role"] == "user"]),
        "session_id": "global-chat"
    }
    try:
        response = requests.post(API_URL, json=payload)
        response.raise_for_status()
        data = response.json()
        # Soportar referencias desde el backend (si las devuelve)
        return {
            "answer": data.get("answer", "❌ No se recibió respuesta."),
            "sources": data.get("sources", [])
        }
    except Exception as e:
        return {"answer": f"❌ Error al conectar con el backend: {e}", "sources": []}

# ========================
# UI Streamlit
# ========================
st.set_page_config(page_title="RAG Chat Público", page_icon="🤖", layout="wide")
st.title("🤖 Soporte Técnico Comunitario")

# Mostrar historial global
for msg in st.session_state.chat_history:
    with st.chat_message(msg["role"]):
        st.markdown(msg["text"])
        # Mostrar referencias si existen
        if "sources" in msg and msg["sources"]:
            st.caption("📖 Referencias: " + ", ".join(msg["sources"]))

# Input de usuario
user_input = st.chat_input("Escribe tu pregunta (visible para todos)...")

if user_input:
    # Añadir pregunta al historial
    st.session_state.chat_history.append({"role": "user", "text": user_input})
    with st.chat_message("user"):
        st.markdown(user_input)

    # Obtener respuesta del backend
    with st.spinner("Pensando..."):
        ai_response = send_question(user_input, st.session_state.chat_history)

    # Añadir respuesta al historial
    st.session_state.chat_history.append({
        "role": "assistant",
        "text": ai_response["answer"],
        "sources": ai_response["sources"]
    })

    with st.chat_message("assistant"):
        st.markdown(ai_response["answer"])
        if ai_response["sources"]:
            st.caption("📖 Referencias: " + ", ".join(ai_response["sources"]))

    # Guardar historial en disco
    with open(HISTORIAL_FILE, "w", encoding="utf-8") as f:
        json.dump(st.session_state.chat_history, f, ensure_ascii=False, indent=2)

# Botón para reiniciar historial global
if st.button("🗑️ Limpiar historial global"):
    st.session_state.chat_history = []
    if os.path.exists(HISTORIAL_FILE):
        os.remove(HISTORIAL_FILE)
    st.experimental_rerun()


Overwriting app_soporte.py


In [11]:
!streamlit run app_soporte.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.219.77:8501

gio: http://localhost:8501: Operation not supported
^C
  Stopping...
